In [28]:
import psycopg2
import pandas as pd

sqluser = 'student'
dbname = 'mimic'
password = 'lipup'
host = 'luolabwkstn01.fsm.northwestern.edu'
port = '5432'
schema_name = 'mimiciii'

con = psycopg2.connect(dbname = dbname, user = sqluser, password = password, host = host, port = port)

In [30]:
query = """
select * from public.pan
"""

pan = pd.read_sql_query(query, con)
pan.head()

,subject_id,hadm_id,icustay_id,intime,outtime,los,dob,dod,dod_hosp,dod_ssn,admittime,dischtime,deathtime,edregtime,edouttime,hospital_expire_flag
0,268,110404,280836,2198-02-14 23:27:38,2198-02-18 05:26:11,3.2490,2132-02-21,2198-02-18,2198-02-18,NaT,2198-02-11 13:40:00,2198-02-18 03:55:00,2198-02-18 03:55:00,2198-02-11 09:41:00,2198-02-11 15:18:00,1
1,269,106296,206613,2170-11-05 11:05:29,2170-11-08 17:46:57,3.2788,2130-09-30,NaT,NaT,NaT,2170-11-05 11:04:00,2170-11-27 18:00:00,NaT,2170-11-05 07:22:00,2170-11-05 12:15:00,0
2,270,188028,220345,2128-06-24 15:05:20,2128-06-27 12:32:29,2.8939,2048-05-26,NaT,NaT,NaT,2128-06-23 18:26:00,2128-06-27 12:31:00,NaT,NaT,NaT,0
3,272,164716,210407,2186-12-25 21:08:04,2186-12-27 12:01:13,1.6202,2119-11-21,NaT,NaT,NaT,2186-12-25 21:06:00,2187-01-02 14:57:00,NaT,NaT,NaT,0
4,273,158689,241507,2141-04-19 06:12:05,2141-04-20 17:52:11,1.4862,2107-08-10,NaT,NaT,NaT,2141-04-19 06:11:00,2141-04-20 17:00:00,NaT,2141-04-19 02:28:00,2141-04-19 09:08:00,0


In [31]:
#df records the 28 days mortality flag and 90 days mortality flag

query = """
with co as (
select subject_id,hadm_id,icustay_id, intime,
(date_part('day', dod::timestamp - intime::timestamp) + 1) 
as mortality 
from public.pan
),
tem as(
select co.*, 
case when mortality <= 28 then 1 else 0 end as mort_28, /*variable name should not start with number*/
case when mortality <= 90 then 1 else 0 end as mort_90
from co where mortality is not null
order by subject_id, hadm_id, icustay_id
)
select * from tem
"""

df = pd.read_sql_query(query, con)
df.head()

,subject_id,hadm_id,icustay_id,intime,mortality,mort_28,mort_90
0,3,145834,211552,2101-10-20 19:10:11,237.0,0,0
1,9,150750,220597,2149-11-09 13:07:02,5.0,1,1
2,11,194540,229441,2178-04-16 06:19:32,212.0,0,0
3,12,112213,232669,2104-08-08 02:08:17,12.0,1,1
4,19,109235,273430,2108-08-05 16:26:09,378.0,0,0


In [32]:
mort_28 = (df['mort_28'] == 1).astype(int).sum(axis=0)

print ('Here is {} icu stays with 28 days mortality.'.format(mort_28))


Here is 7843 icu stays with 28 days mortality.


In [33]:
mort_90 = (df['mort_90'] == 1).astype(int).sum(axis=0)

print ('Here is {} icu stays with 90 days mortality.'.format(mort_90))

Here is 10968 icu stays with 90 days mortality.


In [41]:
query = """
with co as (
select subject_id,hadm_id,icustay_id, intime,
(date_part('day', dod::timestamp - intime::timestamp) + 1) 
as mortality 
from public.pan
),
tem as(
select co.*, 
case when mortality <= 28 then 1 else 0 end as mort_28, /*variable name should not start with number*/
case when mortality <= 90 then 1 else 0 end as mort_90
from co where mortality is not null
order by subject_id, hadm_id, icustay_id
),
tem1 as(
select tem.*, ROW_NUMBER()
            OVER (PARTITION BY hadm_id ORDER BY intime) AS times
from tem where mort_28 = 1
)
select * from tem1
"""
d_28 = pd.read_sql_query(query, con)
d_28.head()

,subject_id,hadm_id,icustay_id,intime,mortality,mort_28,mort_90,times
0,24912,100053,217069,2124-07-14 03:20:29,5.0,1,1,1
1,11728,100061,201906,2178-12-25 14:12:10,2.0,1,1,1
2,9889,100068,204988,2192-01-13 10:53:25,16.0,1,1,1
3,26885,100074,270174,2176-04-09 21:03:47,3.0,1,1,1
4,42255,100087,294404,2126-11-01 17:47:40,5.0,1,1,1


In [43]:
times_28 = (d_28['times'] > 1).astype(int).sum(axis=0)
times_28

677

In [44]:
query = """
with co as (
select subject_id,hadm_id,icustay_id, intime,
(date_part('day', dod::timestamp - intime::timestamp) + 1) 
as mortality 
from public.pan
),
tem as(
select co.*, 
case when mortality <= 28 then 1 else 0 end as mort_28, /*variable name should not start with number*/
case when mortality <= 90 then 1 else 0 end as mort_90
from co where mortality is not null
order by subject_id, hadm_id, icustay_id
),
tem1 as(
select tem.*, ROW_NUMBER()
            OVER (PARTITION BY hadm_id ORDER BY intime) AS times
from tem where mort_90 = 1
)
select * from tem1
"""
d_90 = pd.read_sql_query(query, con)
d_90.head()

,subject_id,hadm_id,icustay_id,intime,mortality,mort_28,mort_90,times
0,68591,100016,217590,2188-05-24 13:07:20,43.0,0,1,1
1,24912,100053,217069,2124-07-14 03:20:29,5.0,1,1,1
2,11728,100061,201906,2178-12-25 14:12:10,2.0,1,1,1
3,9889,100068,204988,2192-01-13 10:53:25,16.0,1,1,1
4,26885,100074,270174,2176-04-09 21:03:47,3.0,1,1,1


In [45]:
times_90 = (d_90['times'] > 1).astype(int).sum(axis=0)
times_90

1169

In [49]:
mort_28 = (df['mort_28'] == 1).astype(int).sum(axis=0)
times_28 = (d_28['times'] > 1).astype(int).sum(axis=0)

print ('{} icu stays with 28 days mortality.'.format(mort_28))
print ('{} patients have more once icu stay within 28 days of their death.'.format(times_28))
mort_90 = (df['mort_90'] == 1).astype(int).sum(axis=0)
times_90 = (d_90['times'] > 1).astype(int).sum(axis=0)

print(' ')

print ('{} icu stays with 90 days mortality.'.format(mort_90))
print ('{} patients have more once icu stay within 90 days of their death.'.format(times_90))

7843 icu stays with 28 days mortality.
677 patients have more once icu stay within 28 days of their death.
 
10968 icu stays with 90 days mortality.
1169 patients have more once icu stay within 90 days of their death.
